In [5]:
import matplotlib.pyplot as plt
import joblib
import numpy as np
import os
import tensorflow as tf
import tensorflow.keras as k
from read_images import read_prof_images, read_our_radar_data
from sklearn.model_selection import train_test_split
from read_images import read_prof_images
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.multiclass import OneVsOneClassifier
from sklearn import metrics
from sklearn.metrics import roc_auc_score

root_dir = os.path.dirname(os.getcwd())

In [2]:
use_prof_samples = False
use_ml = True
loop_models = False

In [6]:
def machine_learning(classifier, ttsData, classifier_name):
    X_train, X_test, y_train, y_test = ttsData
    # Learn the digits on the train subset
    classifier.fit(X_train, y_train)

    joblib.dump(classifier, os.path.join(root_dir, f'model_files/{classifier_name}.pkl'))    # Save the model as a pickle in a file
    model = joblib.load(os.path.join(root_dir, f'model_files/{classifier_name}.pkl'))        # Load the model from the file

    print(f"{classifier} Model")
    # Check the Goodness of Fit (on Train Data)
    print("Goodness of Fit of Model \tTrain Dataset")
    train_accuracy = model.score(X_train, y_train)
    print("Classification Accuracy \t:", train_accuracy, end='\n')

    # Check the Goodness of Fit (on Test Data)
    print("Goodness of Fit of Model \tTest Dataset")
    test_accuracy = model.score(X_test, y_test)
    print("Classification Accuracy \t:", test_accuracy, end='\n')

    # disp = metrics.plot_confusion_matrix(model, X_test, y_test)
    # disp.figure_.suptitle("Confusion Matrix")
    # print(f"Confusion matrix:\n{disp.confusion_matrix}")

    # Use the loaded model to make predictions
    # print("Predcited values: ", model.predict(X_test))
    # print("Actual values:    ", y_test)
    y_prob = model.predict_proba(X_test)
    auc = roc_auc_score(y_test, y_prob, multi_class='ovr')
    print("AUC Score: ", auc, end='\n\n')
    return auc

def deep_learning(ttsData):
    X_train, X_test, y_train, y_test = ttsData
    y_train = tf.one_hot(y_train, y.shape[0])
    y_test = tf.one_hot(y_test, y.shape[0])

    model=k.Sequential()
    model.add(tf.keras.Input(shape=X.shape[1:]))
    model.add(k.layers.Conv2D(32,3,3,padding='valid',
        dilation_rate=(1, 1),
        activation="relu"))
    model.add(k.layers.Flatten())
    model.add(k.layers.Dense(64,activation="relu"))
    model.add(k.layers.Dense(64,activation="relu"))
    model.add(k.layers.Dense(64,activation="relu"))
    model.add(k.layers.Dense(y.shape[0],activation="softmax"))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    history = model.fit(X_train,y_train,epochs=50,batch_size=8)

def choose_dataset():
    if use_prof_samples:
        print("Dataset chosen: Prof's images", end='\n')
        X, y, class_labels = read_prof_images()
        # print(X.shape, y.shape)
    else:
        print("Dataset chosen: Radar images", end='\n')
        X, y, class_labels = read_our_radar_data()
        X = np.abs(X)
        X = np.expand_dims(X, axis=-1)
    if use_ml:  
        X = X.reshape(X.shape[0], -1)
    return X,y, class_labels

def select_pipeline(ttsData, classifiers, classifier_names):
    if use_ml:  
        if loop_models:
            auc_score_dict = {}
            fig, ax = plt.subplots(figsize=(15,5))
            for cf, cf_name in zip(classifiers, classifier_names):
                auc = machine_learning(cf, ttsData, cf_name)
                auc_score_dict[cf_name] = auc
            ax.plot(auc_score_dict.keys(), auc_score_dict.values())
            print(auc_score_dict)
            ax.set_title("Comparison of AUC Score of Classifier Models")
            ax.set_xlabel("Classifier Models")
            ax.set_ylabel("AUC Score")
            plt.show()
        else:
            auc = machine_learning(KNeighborsClassifier(3), ttsData, "Nearest Neighbors") #0.93 
    else:
        deep_learning(ttsData)

In [7]:
classifiers = [
KNeighborsClassifier(3),
SVC(kernel="linear", C=0.025, probability=True),
GaussianProcessClassifier(1.0*RBF(1.0)),
DecisionTreeClassifier(max_depth=5),
RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
MLPClassifier(alpha=1, max_iter=1000),
AdaBoostClassifier(),
GaussianNB(),
QuadraticDiscriminantAnalysis()]
# OneVsOneClassifier(SVC(gamma=0.7, C=1))
classifier_names = ["Nearest Neighbors", "Linear SVM", "Gaussian Process",
    "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
    "Naive Bayes", "QDA"]#"RBF SVM"

X, y, class_labels = choose_dataset()
ttsData = train_test_split(X, y, test_size=.3, random_state=12)
# print(X.shape, y.shape, class_labels)   
select_pipeline(ttsData, classifiers, classifier_names)

Dataset chosen: Radar images
KNeighborsClassifier(n_neighbors=3) Model
Goodness of Fit of Model 	Train Dataset
Classification Accuracy 	: 0.8571428571428571
Goodness of Fit of Model 	Test Dataset
Classification Accuracy 	: 0.7083333333333334
AUC Score:  0.8965919701213819

